# IDM_Masterlist
## What does it do?
#### - Creates a masterfile overview of completed participants and provides payment dates. Also creates simplified csv files, removing unused columns.
## Before you run this code the first time:
#### Step 1. Create a new folder on your Desktop called "idm_files". Save this notebook in that folder. This folder will contain everything relating to this script.
#### Step 2. Download the zipped data file from Pavlovia.
#### Step 3. Extract the "data" folder from the compressed zip. Move unzipped "data" folder to idm_files. 
#### Step 4. Run the cells of this notebook. 
## Input
#### Extracted Pavlovia data files
## Output
#### idm_masterlist.csv
##### - Contains assigned IDM ID#, MTurk WorkerID, date/time of data collection, bonus task, bonus amount, bonus delay, each task response rate, completion code, and Ss payment date
#### 23_IDM_####.csv
##### - Simplified csv data file for each completed Ss

In [1]:
import datetime
import numpy as np
import os
import pandas as pd
import re
import shutil
from glob import glob
from os.path import join

#regular expressions to identify different date patterns (web browsers differences)
date_type1 = re.compile(r"[0-9]{4}-[0-9]{2}-[0-9]{2}", re.IGNORECASE) #2023-03-27
date_type2 = re.compile(r"[0-9]{2}-[0-9]{2}-[0-9]{4}", re.IGNORECASE) #03-27-2023
date_type3 = re.compile(r"[0-9]{2}/[0-9]{2}/[0-9]{4}", re.IGNORECASE) #03/27/2023

In [2]:
def check_empty(filename): #determines if data file is empty 
    lines = ""
    with open(filename, "r") as f:
        for line in f:
            lines += line
    return len(lines) < 10

filepath = os.getcwd() #relative path - current working directory

if os.path.exists(join(filepath, "IDM_simp_files")): #checks to see if simp folder exists
    shutil.rmtree(join(filepath, "IDM_simp_files")) #removes old simp folder and files
    os.makedirs(join(filepath, "IDM_simp_files")) #creates new simp folder
else: #no previous simp folder
    os.makedirs(join(filepath, "IDM_simp_files")) #creates new simp folder

data = {"subID": [], "workerID": [], "DateTime": [], "CompCode": [],  "CPDM": [], "CRDM": [], "CDD":[],  
        "BonusTask": [], "BonusDelay": [], "BONUS_AMT": [], "PAY_DATE":[]} #dictionary containing all data to be saved to IDM masterfile
date_idx = [] #index of date for completed files
date = [] #17 digit number representing date and time Ss file was created ("date")
date_format = [] #standardized date/time format(2023-01-25_15h10.30.241)

pavlovia_files = sorted(glob(join(filepath, "data/*.csv"))) #original Pavlovia csv files
##get date/time for each complete Ss data file, standardize date/time format, and convert date/time to integer##
for i, data_name in enumerate(pavlovia_files): 
    if check_empty(data_name) is False: #files which are not completely empty
        pav_data_df = pd.read_csv(data_name, on_bad_lines='skip') #open csv as Pandas dataframe
        if "completion_code" in list(pav_data_df.keys()): #completed experimental runs
            date_idx.append(i) #save index of current file within all Pavlovia files
            date_fix1 = pav_data_df["date"][0].split("_") #split first value in "date" column [date and time]
            if date_type1.match(pav_data_df["date"][0]): ##date/time formatted as 2023-01-25_15h10.30.241
                date_format.append(pav_data_df["date"][0]) #save datetime to data dict
                a = pav_data_df["date"][0].replace("h", ".").replace("_", ".").replace("-",".").split(".") #replace other separators with, then splits on, "."
                date.append("".join([str(item) for item in a])) #save concatenated .split() output
            elif date_type2.match(pav_data_df["date"][0]): ##date/time format as 02-23-2023_15h10.25.875             
                date_fix2 = date_fix1[0].split("-") #split month, day, year and reassemble to match standard format
                date_fix3 = date_fix2[2] + "-" + date_fix2[0] + "-" + date_fix2[1] + "_" + date_fix1[1] #format to standard date/time
                date_format.append(date_fix3) #save standard date/time to list
                a = date_fix3.replace("h", ".").replace("_", ".").replace("-",".").split(".") #replace other separators with, then splits on, "."
                date.append("".join([str(item) for item in a])) #save concatenated .split() output                      
            else: ##date/time format as 03/27/2023_05h14.47.829
                date_fix2 = date_fix1[0].split("/") #split month, day, year and reassemble to match standard format
                date_fix3 = date_fix2[2] + "-" + date_fix2[0] + "-" + date_fix2[1] + "_" + date_fix1[1] #reformats to standard date/time
                date_format.append(date_fix3) #save standard date/time to list
                a = date_fix3.replace("h", ".").replace("_", ".").replace("-",".").split(".") #replace other separators with, then splits on, "."
                date.append("".join([str(item) for item in a])) #save concatenated .split() output

date_idx = np.array(date_idx) #convert to Numpy array to use .argsort()
date = np.array(date) #convert to Numpy array to use .argsort()
sorted_idx = date_idx[date.argsort()] #sort date_idx by date
sorted_date_format = np.array(date_format)[date.argsort()] #sort date_format by date

for i, good_idx in enumerate(sorted_idx): #iterate through each of the completed Ss data files
    data_df = pd.read_csv(pavlovia_files[good_idx]) #open csv file as Pandas dataframe
    data["subID"].append(str(sorted_date_format[i][2:4])+"_IDM_"+f"{i+1:04}") #records new IDM ID#
    data["workerID"].append(data_df["workerId"][0]) #records MTurk ID#   
    data['DateTime'].append(sorted_date_format[i]) #records datetime   
    data["BonusTask"].append(data_df["idm_bonus_exp"][1039]) #records bonus task
    data["CompCode"].append(int(data_df["completion_code"][1043])) #records completion code
    if "cpdm_bonus_amt" in list(data_df.keys()): #CPDM Bonus Trial
        data["BONUS_AMT"].append(int(data_df["cpdm_bonus_amt"][1040])) #records CPDM bonus amount
        data["BonusDelay"].append(0) #records CPDM bonus delay as 0
        data["PAY_DATE"].append(datetime.datetime(*map(int,data["DateTime"][i][:10].split("-")))) #no delay, gets current date (month/day/year)
    elif "crdm_bonus_amt" in list(data_df.keys()): #CRDM Bonus Trial
        data["BONUS_AMT"].append(int(data_df["crdm_bonus_amt"][1041])) #records CRDM bonus amount
        data["BonusDelay"].append(0) #records CRDM bonus delay as 0
        data["PAY_DATE"].append(datetime.datetime(*map(int,data["DateTime"][i][:10].split("-")))) #no delay, gets current date (month/day/year)
    elif "cdd_bonus_amt" in list(data_df.keys()): #CDD Bonus Trial (correct key)
        data["BONUS_AMT"].append(int(data_df["cdd_bonus_amt"][1042])) #records CDD bonus amount
        data["BonusDelay"].append(int(data_df["cdd_bonus_delay"][1042])) #records CDD bonus delay
        #calculates payment date from current date plus number of delay days
        data["PAY_DATE"].append(datetime.datetime(*map(int,data["DateTime"][i][:10].split("-")))+datetime.timedelta(days=int(data_df["cdd_bonus_delay"][1042])))
    elif "cdd_bonus_amount" in list(data_df.keys()): #CDD Bonus Trial (typo key)
        data["BONUS_AMT"].append(int(data_df["cdd_bonus_amount"][1042])) #records CDD bonus amount
        data["BonusDelay"].append(int(data_df["cdd_bonus_delay"][1042])) #records CDD bonus delay
        #calculates payment date from current date plus number of delay days
        data["PAY_DATE"].append(datetime.datetime(*map(int,data["DateTime"][i][:10].split("-")))+datetime.timedelta(days=int(data_df["cdd_bonus_delay"][1042])))
    #calculates CPDM proportion of response trials     
    data["CPDM"].append(round(len(np.where((data_df["cpdm_trial_type"].values == "task") & ((data_df["cpdm_trial_resp.keys"].values == "q") | (data_df["cpdm_trial_resp.keys"].values == "p") 
    | (data_df["cpdm_trial_resp.keys"].values == "a") | (data_df["cpdm_trial_resp.keys"].values == "l")))[0])/800, 2))
    #calculates CRDM proportion of response trials 
    data["CRDM"].append(round(len(np.where((data_df["crdm_trial_type"].values == "task") & ((data_df["crdm_trial_resp.keys"].values == 1) | (data_df["crdm_trial_resp.keys"].values == 2)))[0])/80, 2))
    #calculates CDD proportion of response trials       
    data["CDD"].append(round(len(np.where((data_df["cdd_trial_type"].values == "task") & ((data_df["cdd_trial_resp.keys"].values == 1) | (data_df["cdd_trial_resp.keys"].values == 2)))[0])/96, 2))

    ##drop unused columns to create simplied CSV##
    simp_df = data_df.drop(columns=["participant", "workerId", "hitId", "assignmentId", "a", "tp_a", "b", "tp_b", "c", "tp_c", "survey1.thisRepN", 
                                    "survey1.thisTrialN", "survey1.thisN", "survey1.thisIndex", "survey1.ran", 
                                    "resps", "resp1", "resp2", "resp3", "resp4", "resp5", "resp6",
                                    "survey2.thisRepN", "survey2.thisTrialN", "survey2.thisN", "survey2.thisIndex", "survey2.ran",
                                    "crdm_pract_trial_resp.keys", "crdm_pract_trial_resp.corr", 
                                    "cdd_pract_trial_resp.keys", "cdd_pract_trial_resp.corr","cpdm_pract_trial_resp.keys"])
    #because some Ss did not respond to specific task practice trials, removing each practice.rt requires its own conditional statement
    if "crdm_pract_trial_resp.rt" in simp_df.keys(): 
        simp_df = simp_df.drop(columns=["crdm_pract_trial_resp.rt"])
    if "cdd_pract_trial_resp.rt" in simp_df.keys():
        simp_df = simp_df.drop(columns=["cdd_pract_trial_resp.rt"])
    if "cpdm_pract_trial_resp.rt" in simp_df.keys():
        simp_df = simp_df.drop(columns=["cpdm_pract_trial_resp.rt"])
    #setup to remove unnecessary Psychopy columns from output
    for key in ["crdm_pract_trials1.thisRepN", "crdm_pract_trials2.thisRepN", "cdd_pract_trials1.thisRepN", "cdd_pract_trials2.thisRepN",
                "cpdm_pract_trials1.thisRepN", "cpdm_pract_trials2.thisRepN", "cpdm_pract_trials3.thisRepN"]:
        task_name = key.split("_")[0] #gets just the task name
        task_num = key[-10] #gets just the loop number
        #removes Psychopy-specific CPDM condition run columns
        if key in data_df.keys() and key in ["cpdm_pract_trials1.thisRepN", "cpdm_pract_trials2.thisRepN", "cpdm_pract_trials3.thisRepN"]:    
            simp_df = simp_df.drop(columns=["condition_runs{0}.thisRepN".format(task_num), "condition_runs{0}.thisTrialN".format(task_num),
                                            "condition_runs{0}.thisN".format(task_num), "condition_runs{0}.thisIndex".format(task_num), 
                                            "condition_runs{0}.ran".format(task_num)])
        #removes Psychopy-specific columns for each loop of all three tasks 
        if key in list(simp_df.keys()):
            simp_df = simp_df.drop(columns=["{0}_pract_trials{1}.thisRepN".format(task_name,task_num), "{0}_pract_trials{1}.thisTrialN".format(task_name,task_num), #practice
                                            "{0}_pract_trials{1}.thisN".format(task_name,task_num), "{0}_pract_trials{1}.thisIndex".format(task_name,task_num),
                                            "{0}_pract_trials{1}.ran".format(task_name,task_num), 
                                            "{0}_trials{1}.thisRepN".format(task_name,task_num), "{0}_trials{1}.thisN".format(task_name,task_num), #task 
                                            "{0}_trials{1}.ran".format(task_name,task_num)]) 
    simp_df.to_csv("IDM_simp_files/"+ str(sorted_date_format[i][2:4])+"_IDM_"+f"{i+1:04}"+".csv") #save simplied data file to csv and label with IDM ID#
    print(str(sorted_date_format[i][2:4])+"_IDM_"+f"{i+1:04}") #print new task-specific subject ID#
                    
masterlist_df = pd.DataFrame(data) #convert dictionary to Pandas DF
masterlist_df.to_csv("IDM_masterlist.csv") #save masterlist DF to csv

23_IDM_0001
23_IDM_0002
23_IDM_0003
23_IDM_0004
23_IDM_0005
23_IDM_0006
23_IDM_0007
23_IDM_0008
23_IDM_0009
23_IDM_0010
23_IDM_0011
23_IDM_0012
23_IDM_0013
23_IDM_0014
23_IDM_0015
23_IDM_0016
23_IDM_0017
23_IDM_0018
23_IDM_0019
23_IDM_0020
23_IDM_0021
23_IDM_0022
23_IDM_0023
23_IDM_0024
23_IDM_0025
23_IDM_0026
23_IDM_0027
23_IDM_0028
23_IDM_0029
23_IDM_0030
23_IDM_0031
23_IDM_0032
23_IDM_0033
23_IDM_0034
23_IDM_0035
23_IDM_0036
23_IDM_0037
23_IDM_0038
23_IDM_0039
23_IDM_0040
23_IDM_0041
23_IDM_0042
23_IDM_0043
23_IDM_0044
23_IDM_0045
23_IDM_0046
23_IDM_0047
23_IDM_0048
23_IDM_0049
23_IDM_0050
23_IDM_0051
23_IDM_0052
23_IDM_0053
23_IDM_0054
23_IDM_0055
23_IDM_0056
23_IDM_0057
23_IDM_0058
23_IDM_0059
23_IDM_0060
23_IDM_0061
23_IDM_0062
23_IDM_0063
23_IDM_0064
23_IDM_0065
23_IDM_0066
23_IDM_0067
23_IDM_0068
23_IDM_0069
23_IDM_0070
23_IDM_0071
23_IDM_0072
23_IDM_0073
23_IDM_0074
23_IDM_0075
23_IDM_0076
23_IDM_0077
23_IDM_0078
23_IDM_0079
23_IDM_0080
23_IDM_0081
23_IDM_0082
23_IDM_0083
23_I

In [11]:
import datetime, time
 
secs = 6986
 
result = datetime.timedelta(seconds = secs)

print(result)

1:56:26


### Playing around with datetime.datetime()

In [36]:
print(data["DateTime"][0])
time1 = data["DateTime"][0].replace("h", ".").replace("_", ".").replace("-",".").split(".")
print(time1)
date = "".join([str(item) for item in time1])
print(date)

2023-02-16_05h14.47.829
['2023', '02', '16', '05', '14', '47', '829']
20230216051447829


In [25]:
date1 = masterlist_df["datetime"].values[0][:10].split("-")
print(date1)
x = datetime.datetime(*map(int, date1))
print(x)
print(x+datetime.timedelta(days=4))

['2023', '01', '25']
2023-01-25 00:00:00
2023-01-29 00:00:00
